In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.project.types.discretization.spatial import ActiveCells
from morpheus.project.types.discretization import SpatialDiscretization
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.project.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=ActiveCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization

SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), col_widths=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], total_width=336.89415472885594, row_heights=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], total_height=277.4944856585935, rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=ActiveCells(shape=(10, 10), data=[ActiveCell(col=0, row=0), ActiveCell(col=0, row

## Setup Time Discretization

In [3]:
from morpheus.project.types.discretization.time.TimeUnit import TimeUnit
from morpheus.project.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.project.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

# Setup Model

In [4]:
from morpheus.project.types.Model import Model

model = Model.new()
model = model.with_updated_time_discretization(time_discretization)
model = model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [5]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [6]:
from morpheus.project.types.boundaries.ConstantHeadObservation import ObservationId, HeadValue, ConstantHeadRawDataItem
from morpheus.project.types.geometry import Point
from morpheus.project.types.boundaries.Boundary import BoundaryName
from morpheus.project.types.layers import LayerId
from morpheus.project.types.boundaries.Boundary import ConstantHeadBoundary, ConstantHeadObservation

layers_list = [LayerId.new()]
constant_head = ConstantHeadBoundary.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[model.layers[0].layer_id],
  observations=[
    ConstantHeadObservation(
      observation_id=ObservationId.new(),
      geometry=Point((13.9223, 50.9662)),
      raw_data=[
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          head=HeadValue.from_float(100),
        ),
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          head=HeadValue.from_float(120),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.to_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:
from morpheus.project.types.boundaries.Boundary import BoundaryCollection

boundary_collection = BoundaryCollection.new()
boundary_collection.add_boundary(constant_head)

model = model.with_updated_boundaries(boundaries=boundary_collection)

## Calculate

In [8]:
from morpheus.project.infrastructure.calculation.engines.modflow_2005.types.Mf2005CalculationEngineSettings import \
  Mf2005CalculationEngineSettings
from morpheus.project.types.calculation.CalculationProfile import CalculationProfile, CalculationEngineType
from morpheus.project.types.calculation.Calculation import Calculation


calculation_engine_settings = Mf2005CalculationEngineSettings.default()
calculation_profile = CalculationProfile.new(CalculationEngineType.MF2005)
calculation = Calculation.new(model=model, calculation_profile=calculation_profile)

In [9]:
calculation = Calculation.from_dict(calculation.to_dict())
calculation.calculation_id

CalculationId(value='8661b2a9-05fc-4542-a132-00f45ce4be65')

In [10]:
from morpheus.project.infrastructure.calculation.services.CalculationService import CalculationService

calculation_service = CalculationService.from_calculation(calculation)
calculation_service.calculate()

Model(model_id=ModelId(value='c57081ee-1675-4ae9-a9ce-45dd5077ca4a'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), col_widths=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], total_width=336.89415472885594, row_heights=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], total_height=277.4944856585935, rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), aff

In [11]:
calculation_service.get_log()

CalculationLog(log=['', '                                  MODFLOW-2005     ', '    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL', '                             Version 1.12.00 2/3/2017                        ', '', ' Using NAME file: c57081ee-1675-4ae9-a9ce-45dd5077ca4a.nam ', ' Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/04/29 23:22:28', '', ' Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.', ' Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/04/29 23:22:28', ' Elapsed run time:  0.017 Seconds', '', '  Normal termination of simulation'])

In [12]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='Calculation finished successfully', files=['c57081ee-1675-4ae9-a9ce-45dd5077ca4a.dis', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.chd', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.list', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.lpf', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.nam', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.bas', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.pcg', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.oc', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.hds'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1, number_of_observations=0), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0, number_of_observations=0), concentration_results=None)

In [13]:
head = calculation_service.read_head(0, 0)

In [14]:
import simplejson as json

json = json.dumps(head, ignore_nan=True)
json

'[[113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [null, 113.33300

In [15]:
calculation_service.read_drawdown(0, 0)

[]

In [16]:
calculation_service.read_budget(0, False)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -0.0018196,
 'TOTAL_OUT': -0.0018196,
 'IN-OUT': -0.0018196,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [17]:
calculation_service.read_budget(0, True)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -4.9852e-06,
 'TOTAL_OUT': -4.9852e-06,
 'IN-OUT': -4.9852e-06,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [18]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='Calculation finished successfully', files=['c57081ee-1675-4ae9-a9ce-45dd5077ca4a.dis', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.chd', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.list', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.lpf', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.nam', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.bas', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.pcg', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.oc', 'c57081ee-1675-4ae9-a9ce-45dd5077ca4a.hds'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1, number_of_observations=0), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0, number_of_observations=0), concentration_results=None)